In [0]:
spark

In [0]:
charge_schema='Transaction_Date date,Transaction_ID string,ItemCode integer,ItemDescription string, NumberOfTransaction integer,Charges double,Country string'

In [0]:

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format("csv").option('header',True).option("inferSchema",True).load(file_location)
df.printSchema()

In [0]:
df.createOrReplaceTempView("charge")
Processed_df=spark.sql("""select substring(timestamp,1,10) as date,`Transaction ID` as Transaction_ID,
          userID,ItemCode,ItemDescription,NumberOfTransaction,Charges,Country
           from charge where userID !=-1 """ )

In [0]:

#Processed_df=Processed_df.filter("Date is not null")
Processed_df.createOrReplaceTempView("charge")
Processed_df.count()

In [0]:
#1.	Charges per Payment/region (total charges)
spark.sql("""  
          select Country, round(sum(Charges),2) as total_charge,count(ItemCode) as total_transactions
          from charge
          group by Country        
                   
          """).show()

In [0]:
#2.	Average charges per transaction

spark.sql(" Select count(Transaction_ID) as total_transaction ,round(sum(Charges)/count(Transaction_ID),2) as avg_charge_per_transaction from charge").show()

In [0]:
#	Top 5 items with highest number of transactions
spark.sql("""
          select ItemCode,count(Transaction_ID) count_of_transaction
          from charge
          group by ItemCode
          order by count_of_transaction desc
          
          """).show(5)

In [0]:
#4.	Top 10 item codes with highest charge
spark.sql("""   
          SELECT ItemCode,round(sum(Charges),2) as total_charge from charge where ItemCode!=-1 group by ItemCode order by total_charge desc limit 10
         
          
          """).show(10)



In [0]:
#5.	User_id for those customer who paid highest charges,and total charges 
spark.sql("""      

          select userID as top_customers,round(sum(Charges),2) as total_charge from charge group by userID order by total_charge desc limit 5  
           """
).show()

In [0]:
# month on month growth of charges 
spark.sql("""
           select 
           date,
           round(sum(Charges),2) as total_charge,
           round(sum(Charges)-lag(sum(Charges)) over (order by date),2) as Daily_growth_rate         
            from charge  where date>='01-07-2024'
            group by date 
            order by date
           """).show()